In [6]:
!pip install openai slack_sdk gspread oauth2client nest_asyncio pyppeteer

In [ ]:
!pip install --upgrade openai

In [10]:
import logging
import openai
import gspread
import base64
import nest_asyncio
import asyncio
import time
from oauth2client.service_account import ServiceAccountCredentials
from pyppeteer import launch
from openai import OpenAI
from slack_sdk import WebClient
from slack_sdk.errors import SlackApiError
import os

# ========== 🛠 Логування ==========
logging.basicConfig(level=logging.INFO, format='[%(levelname)s] %(message)s')
log = logging.getLogger(__name__)

# ========== 🔧 CONFIGURATION ==========
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
SHEET_ID = '1-n-1b-RkuAAT_UaTvkCMxMfni8TBsXzRahh5COvL7Mc'
SERVICE_ACCOUNT_JSON = 'databox-statistics-chatgpt.json'
SCREENSHOT_PATH = 'dashboard.png'
GPT_PROMPT = """
You are a business analyst AI. Analyze the dashboard in the screenshot and return a clear, concise summary in the following format:

1. Key metrics and their performance (briefly).
2. Notable trends or anomalies.
3. Potential business implications or risks.
4. Suggested actions or next steps.

Use a numbered list (1–4), no bullet points, no additional commentary or intro. Keep it simple and readable for business stakeholders.
"""

SLACK_BOT_TOKEN = os.getenv("SLACK_BOT_TOKEN")
slack_client = WebClient(token=SLACK_BOT_TOKEN)
client = OpenAI(api_key=OPENAI_API_KEY)
nest_asyncio.apply()

# ========== 🔐 Google Sheets Access ==========
def get_dashboard_data():
    scope = ["https://spreadsheets.google.com/feeds", "https://www.googleapis.com/auth/drive"]
    creds = ServiceAccountCredentials.from_json_keyfile_name(SERVICE_ACCOUNT_JSON, scope)
    client = gspread.authorize(creds)
    sheet = client.open_by_key(SHEET_ID).sheet1
    return sheet.get_all_records()

# ========== 📸 Screenshot ==========
async def take_screenshot(url, path=SCREENSHOT_PATH):
    browser = await launch(headless=True, args=['--no-sandbox'])
    page = await browser.newPage()
    await page.setViewport({'width': 1920, 'height': 1080})
    await page.goto(url, {'waitUntil': 'networkidle2'})
    await page.screenshot({'path': path, 'fullPage': True})
    await browser.close()
    log.info(f"📸 Screenshot saved: {path}")

# ========== 🧠 GPT-4o Image Analysis ==========
def analyze_with_gpt(image_path):
    with open(image_path, 'rb') as img:
        image_data = base64.b64encode(img.read()).decode("utf-8")

    response = client.chat.completions.create(
        model="gpt-4o",
        messages=[
            {
                "role": "user",
                "content": [
                    {"type": "text", "text": GPT_PROMPT},
                    {"type": "image_url", "image_url": {"url": f"data:image/png;base64,{image_data}"}}
                ]
            }
        ],
        temperature=0.3
    )

    log.info("🧠 GPT analysis completed")
    return response.choices[0].message.content

# ========== 📬 Slack Messaging ==========
def send_direct_message_to_slack_user(user_id, user_name, message, dashboard_url, screenshot_path):
    try:
        cleaned_message = message.replace("**", "").strip()
        full_text = f"📊 *Summary for {user_name}:*\n{cleaned_message}\n🔗 {dashboard_url}"

        conv = slack_client.conversations_open(users=[user_id])
        dm_channel = conv["channel"]["id"]

        slack_client.chat_postMessage(channel=dm_channel, text=full_text)

        with open(screenshot_path, "rb") as file_content:
            slack_client.files_upload(
                channels=dm_channel,
                file=file_content,
                title="Dashboard Screenshot",
                filename="dashboard.png",
                initial_comment="🖼️ Dashboard screenshot:"
            )

        log.info(f"📤 Slack message sent to {user_id}")

    except SlackApiError as e:
        log.error(f"❌ Slack API error for {user_id}: {e.response['error']}")
    except Exception as e:
        log.error(f"❌ General error for {user_id}: {e}")

# ========== 🔁 Main Loop ==========
async def process_dashboards():
    data = get_dashboard_data()
    log.info(f"📥 Loaded {len(data)} dashboard rows")

    for row in data:
        name = row.get("Name")
        urls_raw = row.get("URLs") or ""
        slack_ids_raw = row.get("SlackIDs", "")

        log.info(f"🔄 Processing: {name}")

        urls = [u.strip() for u in urls_raw.split(",") if u.strip()]
        if not urls:
            log.warning(f"⚠️ Skipping {name}: no valid URLs")
            continue

        slack_ids = [x.strip() for x in slack_ids_raw.replace("(", "").replace(")", "").split(",") if x.strip()]
        if not slack_ids:
            log.warning(f"⚠️ Skipping {name}: no Slack recipients")
            continue

        try:
            combined_analysis = ""

            for i, url in enumerate(urls):
                screenshot_path = f"dashboard_{i+1}.png"
                log.info(f"  🌐 URL {i+1}: {url}")

                await take_screenshot(url, path=screenshot_path)
                gpt_result = analyze_with_gpt(screenshot_path)
                gpt_result_clean = gpt_result.replace("**", "").strip()

                combined_analysis += f"\n\n📍 *Page {i+1}:*\n{gpt_result_clean}"

            for user_id in slack_ids:
                send_direct_message_to_slack_user(
                    user_id=user_id,
                    user_name=name,
                    message=combined_analysis,
                    dashboard_url=urls[0],
                    screenshot_path="dashboard_1.png"
                )

            log.info(f"✅ Completed for: {name}")

        except Exception as e:
            log.error(f"❌ Error for {name}: {e}")

        time.sleep(2)

# Run
await process_dashboards()

🔄 Processing: TEST
  📸 Screenshot page 1: https://app.databox.com/datawall/291f85dcd8b16d8787fa49a37b145f5bdbd9a4680b1ac6


[INFO] Starting Chromium download.
INFO:pyppeteer.chromium_downloader:Starting Chromium download.
100%|██████████| 183M/183M [00:00<00:00, 200Mb/s]
[INFO] Beginning extraction
INFO:pyppeteer.chromium_downloader:Beginning extraction
[INFO] Chromium extracted to: /root/.local/share/pyppeteer/local-chromium/1181205
INFO:pyppeteer.chromium_downloader:Chromium extracted to: /root/.local/share/pyppeteer/local-chromium/1181205


❌ Error for TEST: Navigation Timeout Exceeded: 30000 ms exceeded.
🔄 Processing: Cold Emails_Zoho Flagship Projects
  📸 Screenshot page 1: https://app.databox.com/datawall/f02bc56520e7b1f39fa6f571383a3f934583e606842d018?itemId=1909170
❌ Error for Cold Emails_Zoho Flagship Projects: Error code: 401 - {'error': {'message': 'Incorrect API key provided: sk-proj-********************************************************************************************************************************************************1BcA. You can find your API key at https://platform.openai.com/account/api-keys.', 'type': 'invalid_request_error', 'param': None, 'code': 'invalid_api_key'}}
